In [41]:
%load_ext bigdata

The bigdata extension is already loaded. To reload it, use:
  %reload_ext bigdata


In [42]:
%hive_start

In [31]:
%timeout 300

In [94]:
%%hive
DROP TABLE IF EXISTS data;

CREATE TABLE data (
    c1 STRING,
    c2 STRING,
    c3 INT)
ROW FORMAT DELIMITED
FIELDS TERMINATED BY '\t'
LINES TERMINATED BY '\n';
LOAD DATA LOCAL INPATH 'data.tsv' INTO TABLE data;

DROP TABLE IF EXISTS data;
OK
Time taken: 0.464 seconds
CREATE TABLE data (
    c1 STRING,
    c2 STRING,
    c3 INT)
ROW FORMAT DELIMITED
FIELDS TERMINATED BY '\t'
LINES TERMINATED BY '\n';
OK
Time taken: 0.101 seconds
LOAD DATA LOCAL INPATH 'data.tsv' INTO TABLE data;
Loading data to table default.data
OK
Time taken: 1.703 seconds


In [95]:
%%hive
SELECT * FROM data LIMIT 10;

SELECT * FROM data LIMIT 10;
OK
B	1999-08-28	14
E	1999-12-06	12
E	1993-07-21	17
C	1991-02-12	13
E	1995-04-25	16
A	1992-08-22	14
B	1999-06-11	12
E	1993-01-27	13
E	1999-09-10	11
E	1990-05-03	16
Time taken: 0.308 seconds, Fetched: 10 row(s)


In [96]:
%%hive
DROP TABLE IF EXISTS count_letra;
CREATE TABLE count_letra
AS
    SELECT letra, count(1) AS num
    FROM
        (SELECT c1 AS letra FROM data) w
GROUP BY
    letra
ORDER BY
    letra;

DROP TABLE IF EXISTS count_letra;
OK
Time taken: 0.206 seconds
CREATE TABLE count_letra
AS
    SELECT letra, count(1) AS num
    FROM
        (SELECT c1 AS letra FROM data) w
GROUP BY
    letra
ORDER BY
    letra;
Query ID = root_20200203003821_55ae0b3f-09ea-47ce-bf2c-30166efbc4c8
Total jobs = 2
Launching Job 1 out of 2
Number of reduce tasks not specified. Estimated from input data size: 1
In order to change the average load for a reducer (in bytes):
  set hive.exec.reducers.bytes.per.reducer=<number>
In order to limit the maximum number of reducers:
  set hive.exec.reducers.max=<number>
In order to set a constant number of reducers:
  set mapreduce.job.reduces=<number>
Starting Job = job_1580680475277_0016, Tracking URL = http://62101e4655e1:8088/proxy/application_1580680475277_0016/
Kill Command = /usr/local/hadoop/bin/hadoop job  -kill job_1580680475277_0016
Hadoop job information for Stage-1: number of mappers: 1; number of reducers: 1
2020-02-03 00:38:46,257 Stage-1 map = 0%,  re

In [97]:
%%hive
SELECT * FROM count_letra LIMIT 10;

SELECT * FROM count_letra LIMIT 10;
OK
A	12
B	10
C	4
D	1
E	13
Time taken: 0.522 seconds, Fetched: 5 row(s)


In [89]:
%%hive
INSERT OVERWRITE DIRECTORY '/tmp/output'
ROW FORMAT DELIMITED FIELDS TERMINATED BY ','
SELECT * FROM count_letra;

INSERT OVERWRITE DIRECTORY '/tmp/output'
ROW FORMAT DELIMITED FIELDS TERMINATED BY ','
SELECT * FROM count_letra;
Query ID = root_20200203003025_38636cd4-6cf5-4df1-9a6a-6df48b948bf0
Total jobs = 3
Launching Job 1 out of 3
Number of reduce tasks is set to 0 since there's no reduce operator
Starting Job = job_1580680475277_0013, Tracking URL = http://62101e4655e1:8088/proxy/application_1580680475277_0013/
Kill Command = /usr/local/hadoop/bin/hadoop job  -kill job_1580680475277_0013
Hadoop job information for Stage-1: number of mappers: 1; number of reducers: 0
2020-02-03 00:31:00,467 Stage-1 map = 0%,  reduce = 0%
2020-02-03 00:31:18,629 Stage-1 map = 100%,  reduce = 0%, Cumulative CPU 8.15 sec
MapReduce Total cumulative CPU time: 8 seconds 150 msec
Ended Job = job_1580680475277_0013
Stage-3 is selected by condition resolver.
Stage-2 is filtered out by condition resolver.
Stage-4 is filtered out by condition resolver.
Moving data to directory hdfs://0.0.0.0:9000/tmp/output/.hive-staging_

In [90]:
!hdfs dfs -copyToLocal /tmp/output output

In [79]:
!hdfs dfs -ls /tmp

Found 2 items
drwxrwx---   - root supergroup          0 2020-02-02 21:56 /tmp/hadoop-yarn
drwxrwxrwx   - root supergroup          0 2020-02-02 22:12 /tmp/hive


In [102]:
!hadoop fs -rm /tmp/output/*

Deleted /tmp/output/000000_0


In [103]:
!hadoop fs -rmdir /tmp/output

In [104]:
!hdfs dfs -mkdir /tmp/output

In [101]:
%hive_quit